In [4]:
import requests
import io
from io import StringIO


url = "http://www.userpage.fu-berlin.de/soga/300/30100_data_sets/DWD.csv"
s = requests.get(url).text

dwd = pd.read_csv(StringIO(s), sep=";", decimal=",")
dwd

,id,DWD_ID,STATION NAME,FEDERAL STATE,LAT,LON,ALTITUDE,PERIOD,RECORD LENGTH,MEAN ANNUAL AIR TEMP,...,MEAN ANNUAL WIND SPEED,MEAN CLOUD COVER,MEAN ANNUAL SUNSHINE,MEAN ANNUAL RAINFALL,MAX MONTHLY WIND SPEED,MAX AIR TEMP,MAX WIND SPEED,MAX RAINFALL,MIN AIR TEMP,MEAN RANGE AIR TEMP
0,0,1,Aach,Baden-Württemberg,47.8413,8.8493,478.0,1931-1986,55,8.2,...,2.0,67.0,NaN,755.0,2.0,32.5,NaN,39.0,-16.3,9.6
1,1,3,Aachen,Nordrhein-Westfalen,50.7827,6.0941,202.0,1851-2011,160,9.8,...,3.0,67.0,1531.0,820.0,3.0,32.3,30.2,36.0,-10.9,7.3
2,2,44,Großenkneten,Niedersachsen,52.9335,8.2370,44.0,1971-2016,45,9.2,...,2.0,67.0,1459.0,759.0,3.0,32.4,29.9,32.0,-12.6,7.8
3,6,71,Albstadt-Badkap,Baden-Württemberg,48.2156,8.9784,759.0,1986-2016,30,7.4,...,2.0,66.0,1725.0,919.0,2.0,30.2,NaN,43.0,-15.5,8.9
4,8,73,Aldersbach-Kriestorf,Bayern,48.6159,13.0506,340.0,1952-2016,64,8.4,...,1.0,65.0,1595.0,790.0,2.0,33.0,NaN,43.0,-19.2,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,999,6314,Nossen,Sachsen,51.0507,13.3003,308.0,1949-2016,67,8.5,...,3.0,57.0,1605.0,657.0,3.0,32.6,25.5,41.0,-17.8,8.2
595,1009,7135,Aulendorf-Haslach,Baden-Württemberg,47.9655,9.6804,565.0,1950-2004,54,7.8,...,2.0,70.0,1635.0,875.0,3.0,31.1,NaN,42.0,-17.5,8.6
596,1011,7244,Neuglobsow (HM),Brandenburg,53.1424,13.0319,62.0,1967-2004,37,8.2,...,2.0,68.0,1325.0,567.0,2.0,32.0,22.3,34.0,-14.9,8.0
597,1054,14311,Hersdorf-Weißenseifen,Rheinland-Pfalz,50.1505,6.5526,530.0,1958-2005,47,7.3,...,2.0,73.0,1428.0,997.0,3.0,29.7,NaN,38.0,-13.3,7.1
